# DM PPC
## Clément VIGAND - Marie PONTALIER

In [2]:
from config import setup
setup()

In [3]:
## install docplex first with $pip install docplex
from docplex.cp.model import *
from docplex.cp.config import get_default

Création du modèle représentant le problème des couples

In [14]:
def couple_model(Lh, Lf, n):
    mdl = CpoModel(name='couples model')
    
    # variables représentant à qui sont liés les h et les f
    h = mdl.integer_var_list(n, 0, n-1, 'h')
    f = mdl.integer_var_list(n, 0, n-1, 'f')
    
    for i in range(n):
        # garantie les couples dans les 2 sens
        mdl.add(mdl.element(f, h[i]) == i)
        for j in range(n):
            # garantie l'unicité des conjoints
            if j != i:
                mdl.add(h[i] != h[j])
                mdl.add(f[i] != f[j])
        
        # On parcourt toutes les femmes possibles qui ne sont pas la préférée absolue
        for k in range(1, n):
            currentFemme = Lh[i][k]
            # On parcourt toutes les femmes préférées par rapport à currentFemme
            for fPrefIndex in range(k):
                fPref = Lh[i][fPrefIndex]
                # Index de l'homme i dans les préférences de la femme fPref
                indexHomme = Lf[fPref].index(i)
                if indexHomme < n:
                    # Parcours tous les hommes moins bien que i pour fPref
                    for hNotPrefIndex in range(indexHomme+1, n):
                        hNotPref = Lf[fPref][hNotPrefIndex]
                        # Contrainte empêchant les couples bloquants
                        mdl.add(if_then(h[i] == currentFemme, f[fPref] != hNotPref))
    return h, f, mdl
            

Création d'exemple

In [37]:
n = 4
Lh = [[1, 2, 0, 3],
      [3, 0, 2, 1],
      [1, 3, 0, 2],
      [2, 0, 3, 2]]
Lf = [[1, 0, 2, 3],
      [2, 3, 0, 1],
      [0, 2, 3, 1],
      [1, 0, 2, 3]]

h, f, mdl = couple_model(Lh, Lf, n)
lsols = mdl.start_search(trace_log = False)

# Affichage des solutions
for sol in lsols:
    for i in range(n):
        print("(h_{0}, {1})".format(i, f[sol[h[i]]].name))
    print()

(h_0, f_2)
(h_1, f_3)
(h_2, f_1)
(h_3, f_0)



## Optimisation

Ajout de la contrainte liée à la satisfaction des mariages

In [48]:
def add_satisfaction(mdl, Lh, Lf, n, h, f):
    # indices des maries et femmes dans les listes de préférence
    indexesH = mdl.integer_var_list(n, 0, n-1, "i_h")
    indexesF = mdl.integer_var_list(n, 0, n-1, "i_h")
    
    # Maximum de ces indices
    MH, MF = mdl.integer_var_list(2, 0, n-1)
    
    # Associe les indices aux elements
    for i in range(n):
        mdl.add(mdl.element(Lh[i], indexesH[i]) == h[i])
        mdl.add(mdl.element(Lf[i], indexesF[i]) == f[i])
        
    # Maximise MH et MF
    mdl.add(MH == mdl.max(indexesH))
    mdl.add(MF == mdl.max(indexesF))
    # Minimise l'écart entre les 2
    mdl.add(mdl.minimize(abs(MH-MF)))
    return mdl, MF, MH

mdl, MH, MH = add_satisfaction(mdl, Lh, Lf, n, h, f)
sol = mdl.solve()
print(sol[MH])

 ! ----------------------------------------------------------------------------
 ! Minimization problem - 49 variables, 110 constraints
 ! Workers              = 1
 ! Presolve             = Off
 ! SearchType           = DepthFirst
 ! Initial process time : 0.01s (0.01s extraction + 0.00s propagation)
 !  . Log search space  : 98.0 (before), 98.0 (after)
 !  . Memory usage      : 343.8 kB (before), 343.8 kB (after)
 ! Using sequential search.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed            Branch decision
                        0         49                 -
 + New bound is 0
 *             2        8  0.01s               (gap is 100.0%)
 ! ----------------------------------------------------------------------------
 ! Search completed, 1 solution found.
 ! Best objective         : 2 (optimal - effective tol. is 0)
 ! Best bound             : 0
 ! ------------------------------------------------------------